In [1]:
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Sense

m = Container()

E = Set(m, "E", description="employees")
S = Set(m, "S", description="shifts")

a = Parameter(m, "a", domain=[E, S],
                         description="1 if employee is available for shift")
r = Parameter(m, "r", domain=S,
                     description="required staff per shift")
h = Parameter(m, "h", domain=E,
                       description="max shifts per employee")

x = Variable(m, "x", domain=[E, S], type="Binary",
             description="1 if employee works shift")

cover = Equation(m, "cover", domain=S,
                 description="cover staffing requirements")
limit = Equation(m, "limit", domain=E,
                 description="respect individual limits")
avail_cons = Equation(m, "avail_cons", domain=[E, S],
                      description="assign only if available")

cover[S] = Sum(E, x[E, S]) >= r[S]
limit[E] = Sum(S, x[E, S]) <= h[E]
avail_cons[E, S] = x[E, S] <= a[E, S]

obj = Sum((E, S), x[E, S])

model = Model(
    m,
    name="staffing",
    equations=[cover, limit, avail_cons],
    problem="MIP",
    sense=Sense.MIN,
    objective=obj,
)

In [2]:
# Insert data (only now!)

E.setRecords(["Alice", "Bob", "Carol"])
S.setRecords(["Morning", "Afternoon"])

a.setRecords([
    ("Alice", "Morning",   1),
    ("Alice", "Afternoon", 0),
    ("Bob",   "Morning",   1),
    ("Bob",   "Afternoon", 1),
    ("Carol", "Morning",   0),
    ("Carol", "Afternoon", 1),
])

r.setRecords([
    ("Morning",  2),
    ("Afternoon", 1),
])

h.setRecords([
    ("Alice", 1),
    ("Bob",   2),
    ("Carol", 1),
])

model.solve()


,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,3.0,12,7,MIP,CPLEX,0.016


In [3]:
print(x.records)

       E          S  level  marginal  lower  upper  scale
0  Alice    Morning    1.0       1.0    0.0    1.0    1.0
1  Alice  Afternoon    0.0       1.0    0.0    1.0    1.0
2    Bob    Morning    1.0       1.0    0.0    1.0    1.0
3    Bob  Afternoon    1.0       1.0    0.0    1.0    1.0
4  Carol    Morning    0.0       1.0    0.0    1.0    1.0
5  Carol  Afternoon    0.0       1.0    0.0    1.0    1.0
